In [43]:
import yfinance as yf
import pandas as pd
import numpy as np
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import datetime
from datetime import timedelta

In [44]:
#Ticker a evaluar
tickers=["XLF.BA","XLK.BA","XLE.BA","ARKK.BA","QQQ.BA","SPY.BA","MSFT.BA","BYD.BA","TM.BA","XOM.BA","JNJ.BA","PG.BA"]

In [45]:
#Declaramos fechas
fecha_hoy = datetime.date.today()+ timedelta(days=1) #Agregar un día porque la descarga de yfinance es exclusivo en su límite superior.
fecha_hoy_inicio = fecha_hoy - datetime.timedelta(days=180)
#Pasamos a string
fecha_hoy_str=fecha_hoy.strftime("%Y-%m-%d")
fecha_hoy_inicio_str=fecha_hoy_inicio.strftime("%Y-%m-%d")

#Datos de autenticación de mailing
smtp_server = "smtp.gmail.com"
smtp_port = 587
remitente = "tkamada07@gmail.com"
password = "awxt imrp xuvj aosh"  
destinatario = "takeshikamada@hotmail.com"
mensaje = MIMEMultipart()
mensaje["From"] = "BOT Python - Take"
mensaje["To"] = destinatario
mensaje["Subject"] = "Resumen de tendencias"

In [52]:
df_precios_concatenado=pd.DataFrame()
mensajes=[]
for ticker in tickers:
    data = yf.download(ticker, start=fecha_hoy_inicio_str, end=fecha_hoy_str, progress=False)

    # 1) No romper si no hay datos
    if data is None or data.empty:
        mensajes.append(f"{ticker}: sin datos en el rango.")
        continue

    df = data.reset_index().copy()
    df["Ticker"] = ticker
    df["Date"] = pd.to_datetime(df["Date"])
    df = df.sort_values("Date")

    # 2) SMAs con min_periods para evitar señales prematuras
    df["SMA5"]  = df["Close"].rolling(5,  min_periods=5).mean()
    df["SMA50"] = df["Close"].rolling(50, min_periods=50).mean()

    # 3) Estado solo donde haya ambas SMAs
    estado = pd.Series(index=df.index, dtype="object")
    mask_ok = df["SMA5"].notna() & df["SMA50"].notna()
    estado[mask_ok] = np.where(df.loc[mask_ok, "SMA5"] > df.loc[mask_ok, "SMA50"], "Alcista", "Bajista")

    # 4) Tomar los últimos DOS estados válidos
    ult2 = estado.dropna().tail(2)

    if len(ult2) < 2:
        mensajes.append(f"{ticker}: datos insuficientes (necesita ≥ 50 velas).")
        # igual acumulá el df si lo usás después
        df_precios_concatenado = pd.concat([df_precios_concatenado, df], ignore_index=True)
        continue

    prev, curr = ult2.iloc[-2], ult2.iloc[-1]

    if prev == "Alcista" and curr == "Bajista":
        mensajes.append(f"{ticker}: Inicio de tendencia Bajista → VENDER")
    elif prev == "Bajista" and curr == "Alcista":
        mensajes.append(f"{ticker}: Inicio de tendencia Alcista → COMPRAR")
    elif curr == "Bajista":
        mensajes.append(f"{ticker}: Se mantiene Bajista")
    else:
        mensajes.append(f"{ticker}: Se mantiene Alcista")

    df_precios_concatenado = pd.concat([df_precios_concatenado, df], ignore_index=True)


cuerpo = "\n".join(mensajes)

C:\Users\tk\AppData\Local\Temp\ipykernel_1720\2976438899.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=fecha_hoy_inicio_str, end=fecha_hoy_str, progress=False)
C:\Users\tk\AppData\Local\Temp\ipykernel_1720\2976438899.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=fecha_hoy_inicio_str, end=fecha_hoy_str, progress=False)
C:\Users\tk\AppData\Local\Temp\ipykernel_1720\2976438899.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=fecha_hoy_inicio_str, end=fecha_hoy_str, progress=False)
C:\Users\tk\AppData\Local\Temp\ipykernel_1720\2976438899.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=fecha_hoy_inicio_str, end=fecha_hoy_str, progress=False)
C:\Users\tk\AppData\Local\Temp\ipykernel_1720\2976438899.py:4: Futur

In [53]:
print(cuerpo)

XLF.BA: Se mantiene Alcista
XLK.BA: datos insuficientes (necesita ≥ 50 velas).
XLE.BA: Se mantiene Alcista
ARKK.BA: Se mantiene Alcista
QQQ.BA: Se mantiene Alcista
SPY.BA: Se mantiene Alcista
MSFT.BA: Se mantiene Alcista
BYD.BA: sin datos en el rango.
TM.BA: Se mantiene Alcista
XOM.BA: Se mantiene Alcista
JNJ.BA: Se mantiene Alcista
PG.BA: Se mantiene Alcista


# Medias Móviles Simples

Golden Cross y Death Cross

In [47]:
df_precios_concatenado["Date"].max()

Timestamp('2025-09-15 00:00:00')

In [48]:
df_precios_concatenado[
    (df_precios_concatenado["Date"] >= pd.to_datetime(fecha_hoy - timedelta(days=4) ))
]

Price        Date    Close     High      Low     Open   Volume   Ticker  \
Ticker              XLF.BA   XLF.BA   XLF.BA   XLF.BA   XLF.BA            
115    2025-09-12  39800.0  39840.0  39080.0  39260.0   4984.0   XLF.BA   
116    2025-09-15  40060.0  40220.0  39600.0  40200.0  10564.0   XLF.BA   
117    2025-09-15      NaN      NaN      NaN      NaN      NaN   XLK.BA   
233    2025-09-12      NaN      NaN      NaN      NaN      NaN   XLE.BA   
234    2025-09-15      NaN      NaN      NaN      NaN      NaN   XLE.BA   
350    2025-09-12      NaN      NaN      NaN      NaN      NaN  ARKK.BA   
351    2025-09-15      NaN      NaN      NaN      NaN      NaN  ARKK.BA   
467    2025-09-12      NaN      NaN      NaN      NaN      NaN   QQQ.BA   
468    2025-09-15      NaN      NaN      NaN      NaN      NaN   QQQ.BA   
585    2025-09-12      NaN      NaN      NaN      NaN      NaN  MSFT.BA   
586    2025-09-15      NaN      NaN      NaN      NaN      NaN  MSFT.BA   
703    2025-09-12      NaN      NaN      NaN      NaN      NaN    TM.BA   
704    2025-09-15      NaN      NaN      NaN      NaN      NaN    TM.BA   
821    2025-09-12      NaN      NaN      NaN      NaN      NaN   XOM.BA   
822    2025-09-15      NaN      NaN      NaN      NaN      NaN   XOM.BA   
939    2025-09-12      NaN      NaN      NaN      NaN      NaN   JNJ.BA   
940    2025-09-15      NaN      NaN      NaN      NaN      NaN   JNJ.BA   
1057   2025-09-12      NaN      NaN      NaN      NaN      NaN    PG.BA   
1058   2025-09-15      NaN      NaN      NaN      NaN      NaN    PG.BA   

Price      SMA5         SMA50   Close  ...              High      Low  \
Ticker                         XLK.BA  ...   JNJ.BA   JNJ.BA   JNJ.BA   
115     38720.0  35056.500000     NaN  ...      NaN      NaN      NaN   
116     39088.0  35206.700000     NaN  ...      NaN      NaN      NaN   
117         NaN           NaN  8855.0  ...      NaN      NaN      NaN   
233     63840.0  57831.000000     NaN  ...      NaN      NaN      NaN   
234     64520.0  58076.000000     NaN  ...      NaN      NaN      NaN   
350     11126.0   9943.400000     NaN  ...      NaN      NaN      NaN   
351     11316.0  10006.800000     NaN  ...      NaN      NaN      NaN   
467     42144.0  37682.900000     NaN  ...      NaN      NaN      NaN   
468     42660.0  37885.100000     NaN  ...      NaN      NaN      NaN   
585     24212.0  22601.681602     NaN  ...      NaN      NaN      NaN   
586     24568.0  22702.782109     NaN  ...      NaN      NaN      NaN   
703     19202.0  16586.700000     NaN  ...      NaN      NaN      NaN   
704     19244.0  16689.000000     NaN  ...      NaN      NaN      NaN   
821     16120.0  14696.442070     NaN  ...      NaN      NaN      NaN   
822     16324.0  14753.744004     NaN  ...      NaN      NaN      NaN   
939     17140.0  15123.444277     NaN  ...  17530.0  17580.0  17080.0   
940     17244.0  15218.145645     NaN  ...  17660.0  17700.0  17370.0   
1057    15294.0  13871.885879     NaN  ...      NaN      NaN      NaN   
1058    15364.0  13917.387285     NaN  ...      NaN      NaN      NaN   

Price      Open   Volume    Close     High      Low     Open   Volume  
Ticker   JNJ.BA   JNJ.BA    PG.BA    PG.BA    PG.BA    PG.BA    PG.BA  
115         NaN      NaN      NaN      NaN      NaN      NaN      NaN  
116         NaN      NaN      NaN      NaN      NaN      NaN      NaN  
117         NaN      NaN      NaN      NaN      NaN      NaN      NaN  
233         NaN      NaN      NaN      NaN      NaN      NaN      NaN  
234         NaN      NaN      NaN      NaN      NaN      NaN      NaN  
350         NaN      NaN      NaN      NaN      NaN      NaN      NaN  
351         NaN      NaN      NaN      NaN      NaN      NaN      NaN  
467         NaN      NaN      NaN      NaN      NaN      NaN      NaN  
468         NaN      NaN      NaN      NaN      NaN      NaN      NaN  
585         NaN      NaN      NaN      NaN      NaN      NaN      NaN  
586         NaN      NaN  

In [49]:
fecha_hoy

datetime.date(2025, 9, 16)

In [50]:
# Cuerpo del correo
mensaje.attach(MIMEText(cuerpo, "plain"))

In [51]:
try:
    # Conectar al servidor SMTP
    servidor = smtplib.SMTP(smtp_server, smtp_port)
    servidor.starttls()  # Seguridad TLS
    servidor.login(remitente, password)
    servidor.send_message(mensaje)
    servidor.quit()
    print(f"Correo enviado exitosamente {destinatario}  ✅")
except Exception as e:
    print(f"Error al enviar el correo: {e}")

Correo enviado exitosamente takeshikamada@hotmail.com  ✅
